# Evaluation Script

Follow script implement foundamental steps for evaluate a Language translation system using Prometheus Eval like Judge

In [1]:
import torch
import pandas as pd
from PrometheusAPI import PrometheusJudge
from pathlib import Path
from io import StringIO
from utils import jsonline
from sklearn.metrics import cohen_kappa_score

### Prepare Judge

In [2]:
DEVICE = ("cuda" if torch.cuda.is_available() else "cpu")
j = PrometheusJudge(DEVICE)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

### Load Data to evaluate

In [8]:
title = Path("opus-mt-itc-itc(opus).csv")
response = pd.read_csv(title, sep=",", quoting=1, encoding='utf-8')

In [9]:
response.head(3)

,Prompt,Original,Target,Translation(Generated),User_Score,Judge_Score(Prometheus),Judge_Score(Gemini),Judge_Score(GPT)
0,>>ita<< Altressì uno amante chiamando merzé al...,Altressì uno amante chiamando merzé alla sua d...,"Così anche un innamorato, invocando pietà dall...",Un altro amante chiamato merzé sua moglie dice...,1,NaN,2,2
1,>>ita<< A Milano fue ripressa la malvagità d' ...,A Milano fue ripressa la malvagità d' una donn...,"A Milano, nel periodo in cui quest’uomo era si...",A Milano fu respinta la malvagità di una donna...,2,NaN,3,4
2,">>ita<< Nella quale battaglia, certo io ebbi s...","Nella quale battaglia, certo io ebbi sempre ar...","In quella battaglia, ebbi sempre il coraggio d...","In cui battaglia, certo, mi sono sempre arso d...",1,NaN,2,3


### Run Judge and Record the Response

In [10]:
j.prompt(response)

KeyError: 'reference_answer'

In [ ]:
r = j.judge(response)

### Human-Metrics Correlation

In [ ]:
x = []
y = []
cohen_kappa_score(x, y)